## Problem2

In [1]:
from os.path import exists
from google.colab import drive
import os

drive.mount('/content/drive/', force_remount = True)
# change to your own directory
os.chdir("drive/My Drive/Colab_20190413")
!ls -al

Mounted at /content/drive/
total 114865
-rw------- 1 root root 15680000 Apr 13 15:03 binarized_mnist_test.amat
-rw------- 1 root root 78400000 Apr 13 15:12 binarized_mnist_train.amat
-rw------- 1 root root 15680000 Apr 13 15:14 binarized_mnist_valid.amat
-rw------- 1 root root    29106 Apr 20 03:45 hw3_2_0419_from_qiang.ipynb
-rw------- 1 root root    35317 Apr 19 17:55 hw3_2_2_merged_0418.ipynb
-rw------- 1 root root    61366 Apr 20 02:52 hw3_3.ipynb
-rw------- 1 root root  3758307 Apr 15 00:57 model_20190415-005730.pkl
-rw------- 1 root root  3758648 Apr 20 03:27 params.pkl
drwx------ 2 root root     4096 Apr 15 00:48 reconstructed
-rw------- 1 root root     2677 Apr 14 18:52 VAE_Final_Try.ipynb
-rw------- 1 root root    13643 Apr 14 03:31 VAE.ipynb
-rw------- 1 root root   143969 Apr 14 03:59 VAE-xiao.ipynb
-rw------- 1 root root    16657 Apr 18 21:08 VAE-xiao+VALID.ipynb
-rw------- 1 root root    35661 Apr 17 18:05 VAE-ying_0416.ipynb


In [2]:
import numpy as np
import torch.nn as nn
import torch
import random
import math
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import torch.nn.functional as F

cuda = True if torch.cuda.is_available() else False
Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
print(cuda)

True


**Load data**

In [0]:
data_dir = "./"    # Change here if necessary

train_data = np.loadtxt(data_dir + "binarized_mnist_train.amat")
valid_data = np.loadtxt(data_dir + "binarized_mnist_valid.amat")
test_data = np.loadtxt(data_dir + "binarized_mnist_test.amat")

**Model definition**

In [0]:
class Encoder(nn.Module):
    def __init__(self, z_dim = 100):
        super(Encoder, self).__init__()
        self.z_dim = z_dim
        self.encode = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = (3,3)),     # 28 -> 26
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2), # 26 -> 13
            nn.Conv2d(32, 64, kernel_size = (3,3)),    # 13 -> 11
            nn.ELU(),
            nn.AvgPool2d(kernel_size = 2, stride = 2), # 11 -> 5
            nn.Conv2d(64, 256, kernel_size = (5,5)),   # 5  -> 1
            nn.ELU(),
        )
        self.linear1 = nn.Linear(in_features = 256, out_features = 2*z_dim)
        
    def forward(self, x):                          # [batch_size, 1, 28, 28]
        q = self.encode(x)                         # [batch_size, 256, 1, 1]
        q = self.linear1(q.view(q.size(0), -1))    # [batch_size, 200]
        mu, log_var = q[:,:z_dim], q[:, z_dim: ]   # [batch_size, 100] 
        return mu, log_var
    
class Decoder(nn.Module):
    def __init__(self, z_dim = 100):
        super(Decoder, self).__init__()
        self.z_dim = z_dim
        self.linear2 = nn.Linear(in_features = z_dim, out_features = 256)
        self.decode = nn.Sequential(
            nn.ELU(),
            nn.Conv2d(256, 64, kernel_size=5, padding=4), 
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),#mode='bilinear'
            nn.Conv2d(64, 32, kernel_size=3, padding=2),
            nn.ELU(),
            nn.UpsamplingBilinear2d(scale_factor=2),#mode='bilinear'
            nn.Conv2d(32, 16, kernel_size=3, padding=2),
            nn.ELU(),
            nn.Conv2d(16, 1, kernel_size=3, padding=2)
        )
        
    def forward(self, z):
        x_ = self.linear2(z).view(z.size(0),-1, 1,1) # [batch_size, 256, 1, 1]
        x_ = self.decode(x_)                       #[batch_size, 1, 28, 28] 
        return x_

class VAE(nn.Module):
    def __init__(self, z_dim = 100):
        super(VAE, self).__init__()
        self.encoder = Encoder(z_dim = z_dim)
        self.decoder = Decoder(z_dim = z_dim)
        
    def reparameterize(self, mu, log_var):
        std = torch.exp(0.5 * log_var)
        eps = torch.randn_like(std)
        return mu + torch.mul(eps, std)
    
    def forward(self, x):
        mu, log_var = self.encoder(x)
        z = self.reparameterize(mu, log_var)
        x_ = self.decoder(z)
        return x_, mu, log_var
    
def compute_elbo(recon_x, x, mu, log_var):    
    # compute log p_{\theta} (x)
    recon_x = recon_x.view(recon_x.size(0), -1)
    x = x.view(x.size(0), -1)
    
    # convert to the probabilities of each pixel to be 1, [batch_size, 1]
    output_px = torch.sigmoid(recon_x) 
    
    # compute the log probabilities of 
    # each reconstructed pixel value = original binary value
    # this is the binary cross entropy
    log_px = torch.log(output_px * x + (1- output_px) * (1-x))
    
    # log prob for a reconstructed image is original image x, [batch_size, 1]
    log_px = torch.sum(log_px, dim = 1, keepdim = True)
    log_px = torch.mean(log_px) # mean over batch_size samples
    # BCE = -1 * log_px         # relationship between BCE and lop_px
    
    # [batch_size, 1]
    KLD = 0.5 * torch.sum((-1 - log_var + torch.pow(mu, 2) + torch.exp(log_var)), 
                          dim = 1, keepdim = True)
    KLD = torch.mean(KLD) # mean over batch_size samples
    # mean elbo over batch_size samples, zero dimension
    elbo = log_px - KLD
    return elbo, -log_px, KLD
    
def compute_loss(recon_x, x, mu, log_var):
    batch_size = recon_x.size(0) # 
    # keep data and dimension, [batch_size, 1, 28, 28]
    BCE = F.binary_cross_entropy_with_logits(recon_x, x, reduction="none") 
    # sum all losses(of pixels) in one image, [batch_size, 1]
    BCE = torch.sum(BCE.view(batch_size, -1), dim = 1, keepdim = True)
    # compute average BCE over batch_size, zero dimension, []
    BCE = torch.mean(BCE) 
    # [batch_size, 1]
    KLD = 0.5 * torch.sum((-1 - log_var + torch.pow(mu, 2) + torch.exp(log_var)), 
                          dim = 1, keepdim = True)
    # average over batch_size, zero dimensiona, []
    KLD = torch.mean(KLD)
    return BCE + KLD, BCE, KLD

def data_loader(data, batch_size = 64):
    """
    params
        data: np.array, [sample_size, 784]
    """
    image_data = torch.unsqueeze(Tensor(data.reshape(-1, 28, 28)), 1)
    return DataLoader(image_data, batch_size = batch_size, shuffle = True)  

**Training**

In [5]:
batch_size = 64
z_dim = 100
vae = VAE(z_dim = z_dim)
if cuda:
    vae.cuda()
    
optimizer = torch.optim.Adam(vae.parameters(), lr = 3e-4)

train_loader = data_loader(train_data, batch_size = batch_size)
valid_loader = data_loader(valid_data, batch_size = batch_size)
test_loader =  data_loader(test_data, batch_size = batch_size)

max_epochs = 20

for epoch in range(max_epochs):
    vae.train()
    for idx, x in enumerate(train_loader):
        recon_x, mu, log_var = vae(x)
        
        # both methods work
        #loss, bce, kld = compute_loss(recon_x, x, mu, log_var)
        elbo, bce, kld = compute_elbo(recon_x, x, mu, log_var)
        loss = -1 * elbo
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if (idx+1) % 100 == 0:
            print ("Epoch[{}/{}], Step [{:5}], loss: {:.4f} BCE {:.4f}, KLD: {:.4f}"
                   .format(epoch+1, max_epochs, idx+1, 
                           -elbo.item(), bce.item(), kld.item()))
     
    vae.eval()
    val_elbo, val_loss, val_size = 0.0, 0.0, 0
    #val_size = len(valid_loader)
    for idx, x in enumerate(valid_loader):
        batch_size = x.shape[0]
        recon_x, mu, log_var = vae(x)
        loss, _, _ = compute_loss(recon_x, x, mu, log_var)
        elbo, _, _ = compute_elbo(recon_x, x, mu, log_var)
        val_elbo += elbo.item() * batch_size
        val_loss += loss.item() * batch_size
        val_size += batch_size
        
    val_loss /= val_size
    val_elbo /= val_size
    print("== Epoch[{}/{}]: elbo on valid set: {:.4f} ==\n"
          .format(epoch+1, max_epochs, val_elbo))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Epoch[1/20], Step [  100], loss: 213.6074 BCE 203.2597, KLD: 10.3477
Epoch[1/20], Step [  200], loss: 173.4395 BCE 158.4358, KLD: 15.0037
Epoch[1/20], Step [  300], loss: 175.4763 BCE 157.2164, KLD: 18.2599
Epoch[1/20], Step [  400], loss: 158.8159 BCE 138.2292, KLD: 20.5866
Epoch[1/20], Step [  500], loss: 142.7286 BCE 122.5219, KLD: 20.2068
Epoch[1/20], Step [  600], loss: 143.6062 BCE 122.1241, KLD: 21.4821
Epoch[1/20], Step [  700], loss: 138.2079 BCE 114.8305, KLD: 23.3774
== Epoch[1/20]: elbo on valid set: -136.1425 ==

Epoch[2/20], Step [  100], loss: 137.2505 BCE 114.2415, KLD: 23.0090
Epoch[2/20], Step [  200], loss: 130.9104 BCE 107.5478, KLD: 23.3626
Epoch[2/20], Step [  300], loss: 126.2803 BCE 102.8191, KLD: 23.4612
Epoch[2/20], Step [  400], loss: 125.6329 BCE 101.7326, KLD: 23.9004
Epoch[2/20], Step [  500], loss: 113.6006 BCE 90.2551, KLD: 23.3454
Epoch[2/20], Step [  600], loss: 119.7551 BCE 95.4663, KLD: 24.2888
Epoch[2/20], Step [  700], loss: 115.6446 BCE 92.5809, K

**Save model parameters**

In [0]:
import os.path

if os.path.isfile('params.pkl'):
    os.remove('params.pkl') 
    
torch.save(vae.state_dict(),  'params.pkl')

**Evaluating**

In [0]:
from torch.distributions.normal import Normal

#M = 50  #batch_size
D, K, L = 784, 200, 100

def sampling(mu, log_var, k = K):
    sigma = torch.exp(0.5 * log_var)
    m = Normal(mu, sigma)
    z = torch.squeeze(m.sample(torch.Size([k])), 1)
    return z

def log_normal_densities(z, mu, log_var):
    log_prob_density = None
    sigma = torch.exp(0.5 * log_var)
    m = Normal(mu, sigma)
    log_probs = m.log_prob(z)
    return torch.sum(log_probs, dim = 1, keepdim = True) # [K, 1]

def log_probs_reconstruct(decoder, x_i, z):
    k = z.size(0)
    recon_xs_i = decoder(z).view(k, -1)            # flatten [K, 784]
    recon_xs_i = torch.sigmoid(recon_xs_i)      # conver to probabilities
    x_i = x_i.view(1, -1)
    xs_i = x_i.repeat(k, 1)                                  # copy original x to K times
    # compare all 784 
    log_p_theta = torch.log(recon_xs_i * xs_i + (1- recon_xs_i) * (1-xs_i))
    return torch.sum(log_p_theta, dim = 1, keepdim = True)  # [K, 1]

def generate_Z(encoder, x, k = K):
    M = x.size(0)
    x = x.view(M, 1, 28, 28)
    mu, log_var = encoder(x)     # [batch_size, L]
    L = mu.size(1)
    Z = sampling(mu, log_var, k)
    Z = torch.transpose(Z, 0, 1)
    return Z
    
def evaluate_log_likelihood(model, x, Z):
    encoder = model.encoder
    decoder = model.decoder
    mini_batch_size = x.size(0) 
    x = x.view(mini_batch_size, -1) 
    K = Z.size(1)          
    log_p = [0 for i in range(mini_batch_size)]
    for i in range(mini_batch_size):
        x_i = x[i, :]          # [M, D]
        z = Z[i, :, :]          # [K, L]
        mu, log_var = encoder(x_i.view(1, 1, 28, 28))    # [1, L], [1, L]
        mu0, log_var0 = torch.zeros(1, L), torch.zeros(1, L)
        if cuda:
            mu0 = mu0.cuda()
            log_var0 = log_var0.cuda()
        log_p_z = log_normal_densities(z, mu0, log_var0) 
        log_q_z = log_normal_densities(z, mu, log_var) 
        log_p_theta_x_i = log_probs_reconstruct(decoder, x_i, z) 
        
        # LogSumExp trick for underflow
        log_p_for_sum = log_p_theta_x_i + log_p_z - log_q_z
        max_log_p = torch.max(log_p_for_sum)
        log_p_for_sum1 = log_p_for_sum - max_log_p
        log_1k = torch.log(torch.tensor([1./K]))
        log_p_xi = log_1k + max_log_p + torch.log(torch.sum(torch.exp(log_p_for_sum1)))
        log_p[i] = log_p_xi.item()
    
    return log_p

In [0]:
encoder = vae.encoder
decoder = vae.decoder

In [9]:
print(x.shape)
Z = generate_Z(encoder, x, k = 200)
print(Z.shape)

torch.Size([16, 1, 28, 28])
torch.Size([16, 200, 100])


**Evaluate with ELBO**

In [0]:
model = VAE(z_dim = z_dim)
if cuda:
    model.cuda()
model.load_state_dict(torch.load('params.pkl'))
model.eval()

def calculate_elbo(model, data_loader):
    data_elbo, data_loss, data_size = 0.0, 0.0, 0
    for idx, x in enumerate(data_loader):
        batch_size = x.shape[0]
        recon_x, mu, log_var = model(x)
        loss, _, _ = compute_loss(recon_x, x, mu, log_var)
        elbo, _, _ = compute_elbo(recon_x, x, mu, log_var)
        data_elbo += elbo.item() * batch_size
        data_loss += loss.item() * batch_size
        data_size += batch_size
        
    data_loss /= data_size
    data_elbo /= data_size
    return data_elbo

In [11]:
print("elbo on valid set = ", calculate_elbo(model, valid_loader))
print("elbo on test set = ", calculate_elbo(model, test_loader))

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


elbo on valid set =  -94.33704245605469
elbo on test set =  -93.69906392822266


**Evaluate with log-likelihood**

In [0]:
def  evaluate(model, data_loader):
    log_px = []
    
    for idx, x in enumerate(test_loader):
        Z = generate_Z(model.encoder, x, k = 200)
        log_px.extend(evaluate_log_likelihood(model, x, Z))

    mean_log_px = 0.0
    for i in range(len(log_px)):
        mean_log_px += log_px[i]
    mean_log_px /= len(log_px)
    
    return mean_log_px, log_px

In [13]:
#valid_loader = data_loader(valid_data, batch_size = batch_size)
#test_loader =  data_loader(test_data, batch_size = batch_size)

valid_mean_log_px,  valid_log_px = evaluate(model, valid_loader)
test_mean_log_px,  test_log_px = evaluate(model, test_loader)

#print(valid_log_px)
print("Mean log-likelihood on valid set = ", valid_mean_log_px)
#print(test_log_px)
print("Mean log-likelihood on test set = ", test_mean_log_px)

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/upsampling.py:129: UserWarning: nn.UpsamplingBilinear2d is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.{} is deprecated. Use nn.functional.interpolate instead.".format(self.name))


Mean log-likelihood on valid set =  -88.38922895717621
Mean log-likelihood on test set =  -88.38320319671631
